# HyperLogLog

In [1]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import Column
from pyspark.sql.functions import col, trim, regexp_replace, hll_sketch_estimate, hll_sketch_agg

In [3]:
spark = SparkSession.builder.master("local").appName("demo").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

24/10/14 08:30:09 WARN Utils: Your hostname, Matthews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.188 instead (on interface en0)
24/10/14 08:30:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/14 08:30:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 63166)
Traceback (most recent call last):
  File "/opt/miniconda3/envs/pyspark-351/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/miniconda3/envs/pyspark-351/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/m

In [4]:
df = spark.createDataFrame([
    (1, "a"), 
    (2, "a"),
    (3, "a"),
    (4, "b"),
], ["number", "letter"])

In [15]:
df.show()

+------+------+
|number|letter|
+------+------+
|     1|     a|
|     2|     a|
|     3|     a|
|     4|     b|
+------+------+



In [13]:
hlls = df.agg(
    hll_sketch_agg(df.number).alias("numbers_hll"), 
    hll_sketch_agg(df.letter).alias("letters_hll")
)

In [14]:
hlls.show()

+--------------------+--------------------+
|         numbers_hll|         letters_hll|
+--------------------+--------------------+
|[02 01 07 0C 03 0...|[02 01 07 0C 03 0...|
+--------------------+--------------------+



In [18]:
hlls.select(
    hll_sketch_estimate(hlls.numbers_hll).alias("distinct_numbers"), 
    hll_sketch_estimate(hlls.letters_hll).alias("distinct_letters")
).show()

+----------------+----------------+
|distinct_numbers|distinct_letters|
+----------------+----------------+
|               4|               2|
+----------------+----------------+

